In [15]:
import numpy as np
import random
import MySQLdb
import scipy.spatial
import pickle
from math import sqrt
import sys

In [16]:
# Database Connection Setup

db = MySQLdb.connect(host="localhost",user="root",passwd="jalaz",db="kitabghar")
cur = db.cursor()

In [90]:
# Hash the UserId into indices for better matrix calculations

hashUsers = {}
reverseHashUsers = []
query = "SELECT userid from users"
cur.execute(query)

for i,row in enumerate(cur.fetchall()):
    hashUsers[row[0]] = i
    reverseHashUsers.append(row[0])
users = len(hashUsers)
print("Total Subscribers: {}".format(users))

Total Subscribers: 14


In [91]:
# Hash the BookId into indices for better matrix calculations

hashBooks = {}
reverseHashBooks = []
query = "SELECT bookid from books"
cur.execute(query)

for i,row in enumerate(cur.fetchall()):
    hashBooks[row[0]] = i
    reverseHashBooks.append(row[0])
books = len(hashBooks)
print("Total Books: {}".format(books))

Total Books: 130


In [19]:
###### 1 : Popularity Based Recommendation System #####

In [31]:
purchaseBooksCounter = []
query = "SELECT bookid, count(*) from shelfs GROUP BY bookid"
cur.execute(query)

for row in cur.fetchall():
    purchaseBooksCounter.append((row[1],row[0]))

def firstElement(element):
    return element[0]
purchaseBooksCounter.sort(reverse = True, key = firstElement)

print("Most Popular Books(Sales):")
for i,book in enumerate(purchaseBooksCounter):
    if i>=7:
        print(book[1])
        break
    print(book[1],end="-|-")

Most Popular Books(Sales):
PROG1023-|-NOV1046-|-ACAD1004-|-ACAD1009-|-ACAD1030-|-ACAD1042-|-NOV1015-|-NOV1017


In [32]:
ratingCalculatorBooks = []
query = "SELECT bookid, sum(rating), count(*) from reviews GROUP BY bookid"
cur.execute(query)

for row in cur.fetchall():
    normalizedRating = round((int(row[1])/int(row[2])),2)
    ratingCalculatorBooks.append((normalizedRating, row[0], row[2]))
    
ratingCalculatorBooks.sort(reverse = True, key = firstElement)

print("Most Popular Books(Ratings):")
for i,book in enumerate(ratingCalculatorBooks):
    if i>=7:
        print(book[1])
        break;
    print(book[1], end="-|-")

Most Popular Books(Ratings):
NOV1013-|-NOV1021-|-PROG1031-|-ACAD1029-|-NOV1037-|-NOV1047-|-ACAD1002-|-ACAD1035


In [33]:
#####  2 - Item-Item Based Collaborative Filtering Recommender System   #####

In [113]:
# Grab Dynamic Ratings directly from the Database

ratingData = []
query = "SELECT userid,bookid,rating from reviews"
cur.execute(query)

for row in cur.fetchall():
    ratingData.append((hashUsers[row[0]], row[0], hashBooks[row[1]], row[1], row[2]))
    
print("Done: Dynamic Rating Grabbed")

Done: Dynamic Rating Grabbed


In [36]:
# Grab the total number of Users who have rated

query = "SELECT count(*) from reviews GROUP BY userid"
cur.execute(query)
print(len(cur.fetchall()))

5


In [37]:
# Grab the total number of Books Rated

query = "SELECT count(*) from reviews GROUP BY bookid"
cur.execute(query)
print(len(cur.fetchall()))

128


In [111]:
# Create the Data Matrix

dataMatrix = np.zeros((users,books))
for entry in ratingData:
    dataMatrix[entry[0]][entry[2]] = entry[4]
    
print("Constructed: Data Matrix")
print(dataMatrix)

Constructed: Data Matrix
[[ 4.  4.  0. ...,  4.  0.  0.]
 [ 3.  4.  0. ...,  1.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 ..., 
 [ 5.  2.  2. ...,  0.  0.  2.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]]


In [105]:
# Getting Similarity Matrix for the Books

item_similarity_cosine_books = np.zeros((books,books))
for book1 in range(books):
    for book2 in range(books):
        if np.count_nonzero(dataMatrix[:, book1]) and np.count_nonzero(dataMatrix[:, book2]):
            similarity_value = round((1 - scipy.spatial.distance.cosine(dataMatrix[:, book1], dataMatrix[:, book2])),3)
            item_similarity_cosine_books[book1][book2] = similarity_value
            
print("Done: Constructed Similarity Matrix")

Done: Constructed Similarity Matrix


In [107]:
# Gather the List of Books which User has already Read & Rated

userid = 'jalaz.kumar'
alreadyBooksRead = []
query = "SELECT bookid from shelfs WHERE userid = '{uid}'".format(uid = userid)
cur.execute(query)

alreadyRecommended = {}

for row in cur.fetchall():
    alreadyBooksRead.append(row[0])
    alreadyRecommended[row[0]] = True

print("Read & Rated Books:", alreadyBooksRead)

Read & Rated Books: ['NOV1015', 'PROG1009', 'PROG1023', 'NOV1046']


In [109]:
# Generating the recommendations for the User

recommendations = []
for book in alreadyBooksRead:
    for i,simValue in enumerate(item_similarity_cosine_books[hashBooks[book]]):
        recommendations.append((reverseHashBooks[i],float(simValue)))

def secondElement(element):
    return element[1]

recommendations.sort(reverse = True, key=secondElement)

finalRecommended = []
booksRecommenderCounter = 0
for entry in recommendations:
    if booksRecommenderCounter>=23:
        break;
        
    if entry[1] != 1 and entry[0] not in alreadyRecommended:
        finalRecommended.append(entry[0])
        alreadyRecommended[entry[0]] = True
        booksRecommenderCounter += 1
        
print("Recommrndations Generated:", finalRecommended)

Recommrndations Generated: ['ACAD1029', 'NOV1037', 'NOV1048', 'NOV1020', 'ACAD1015', 'PROG1021', 'PROG1012', 'ACAD1025', 'ACAD1036', 'NOV1011', 'NOV1036', 'ACAD1002', 'NOV1000', 'NOV1005', 'ACAD1037', 'PROG1028', 'PROG1002', 'ACAD1014', 'ACAD1044', 'ACAD1003', 'ACAD1040', 'PROG1010', 'PROG1000']
